In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Ant Colony Optimization Algorithm
class ACO:
    def __init__(self, cost_func, heuristic_func, n_ants=10, n_best=5, n_iterations=100, decay=0.5, alpha=1, beta=1):
        self.cost_func = cost_func
        self.heuristic_func = heuristic_func
        self.n_ants = n_ants
        self.n_best = n_best
        self.n_iterations = n_iterations
        self.decay = decay
        self.alpha = alpha
        self.beta = beta
        self.n_values = 2 ** 8
        self.pheromone = np.ones((self.cost_func.n_params, self.n_values))

    def _update_pheromone(self, delta, best):
        self.pheromone = (1 - self.decay) * self.pheromone + delta * best

    def optimize(self):
        best_cost = float('inf')
        best_solution = None
        for i in range(self.n_iterations):
            solutions = []
            for j in range(self.n_ants):
                solution = []
                for k in range(self.cost_func.n_params):
                    prob = []
                    for l in range(self.n_values):
                        prob.append(self.pheromone[k][l] ** self.alpha * (1.0 / self.heuristic_func(k, l)) ** self.beta)
                    prob = prob / sum(prob)
                    solution.append(np.random.choice(range(self.n_values), p=prob))
                solutions.append(solution)
            costs = []
            for solution in solutions:
                costs.append(self.cost_func(solution))
            best_index = np.argmin(costs)
            if costs[best_index] < best_cost:
                best_cost = costs[best_index]
                best_solution = solutions[best_index]
            sorted_costs = np.argsort(costs)
            delta = np.zeros_like(self.pheromone)
            for j in range(self.n_best):
                index = sorted_costs[j]
                for k in range(self.cost_func.n_params):
                    delta[k][solutions[index][k]] += 1.0 / costs[index]
            self._update_pheromone(delta, 1.0 / best_cost)
        return best_solution

# Neural Network Model
class Net(nn.Module):
    def __init__(self, input_size, output_size):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(input_size, 32)
        self.fc2 = nn.Linear(32, 16)
        self.fc3 = nn.Linear(16, output_size)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = torch.sigmoid(self.fc3(x))
        return x

# Cost Function for ACO Algorithm
class CostFunction:
    def __init__(self, model, criterion, optimizer, train_loader):
        self.model = model
        self.criterion = criterion
        self.optimizer = optimizer
        self.train_loader = train_loader
        self.n_params = sum(p.numel() for p in model.parameters())
        self.n_values = 2 ** 8

    def __call__(self, solution):
        params = []
        for p in solution:
            params.append((p - (self.n_values - 1) / 2) / ((self.n_values - 1) / 2))
        i = 0
        for p in model.parameters():
            size = p.numel()
            value = params[i:i+size]
            value = torch.tensor(value).view(p.size())
            p.data.copy_(value)
            i += size

        cost = 0.0
        for data in train_loader:
            inputs, labels = data
            outputs = model(inputs)
            loss = criterion(outputs.view(-1), labels.float())
            cost += loss.item()
        
        return cost / len(train_loader)

# Load Data from CSV File
#data = pd.read_csv('Bank_Personal_Loan_Modelling.csv')
dataframe=pd.read_csv('Bank_Personal_Loan_Modelling.csv')
data = dataframe.drop(["ID","Age","ZIP Code"],axis = 1)

# Split the data into features (X) and labels (y)
X = dataframe.drop('Personal Loan', axis=1).values.astype('float32')
y = dataframe['Personal Loan'].values.astype('float32')


#X=dataframe.iloc[:,:-1].values.astype('float32')
#y=dataframe.iloc[:,-1].values.astype('float32')

# Split Data into Training and Test Sets
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

# Scale Data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


# Convert Data to PyTorch Tensors
X_train=torch.from_numpy(X_train)
y_train=torch.from_numpy(y_train)
X_test=torch.from_numpy(X_test)
y_test=torch.from_numpy(y_test)

# Create Data Loaders
train_data=TensorDataset(X_train,y_train)
train_loader=DataLoader(train_data,batch_size=10)

# Create Neural Network Model
model=Net(input_size=X.shape[1],output_size=1)

# Set Loss Function and Optimizer
criterion=nn.BCELoss()
optimizer=optim.Adam(model.parameters(),lr=0.001)

# Train Neural Network using ACO Algorithm
cost_func=CostFunction(model,criterion,optimizer,train_loader)
def heuristic_func(param_index, value_index):
    return 1.0

aco = ACO(cost_func, heuristic_func, n_iterations=10)
best_solution=aco.optimize()

# Set Weights of Neural Network to Best Solution Found by ACO Algorithm
params=[]
for p in best_solution:
    params.append((p-(aco.n_values-1)/2)/((aco.n_values-1)/2))
i=0
for p in model.parameters():
    size=p.numel()
    value=params[i:i+size]
    value=torch.tensor(value).view(p.size())
    p.data.copy_(value)
    i+=size

# Test Neural Network Model
with torch.no_grad():
    y_pred=model(X_test)
    y_pred=(y_pred>0.5).float()
    accuracy=(y_pred==y_test).float().mean()
    print('Accuracy:',accuracy.item())

Accuracy: 0.8831999897956848
